# RLDT


## Step 1: Import the necessary libraries:


In [10]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

## Step 2: Define the environment:


#### ALL THE DEVICES


In [11]:
import ast

devices = pd.read_csv("devices.csv")
devices["voltages_frequencies"] = devices["voltages_frequencies"].apply(lambda x: ast.literal_eval(x))
devices["capacitance"] = devices["capacitance"].apply(
    lambda x: ast.literal_eval(x)
)
devices["occupied_cores"] = devices["occupied_cores"].apply(
    lambda x: ast.literal_eval(x)
)
devices["powerIdle"] = devices["powerIdle"].apply(
    lambda x: ast.literal_eval(x)
)
devices["acceptableTasks"] = devices["acceptableTasks"].apply(
    lambda x: ast.literal_eval(x)
)
devices = devices.drop(["Unnamed: 0"],axis=1)
# devices

### Step 2.2: Application


#### _ALL THE TASKS_


In [12]:
tasks = pd.read_csv("tasks.csv")
tasks = tasks.drop(["Unnamed: 0"],axis=1)
tasks = tasks.set_index("id")
tasks

,job,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,,
t104508,8637,"['t104500', 't104502', 't104506', 't104507']",4,3,0,6174652,1850452,3299959,READY
t54855,4495,"['t54848', 't54850', 't54851', 't54852']",3,1,0,6485964,9119114,10992002,READY
t33132,2708,"['t33129', 't33130', 't33131']",6,3,0,9624990,1420604,8150017,READY
t108587,8982,"['t108583', 't108585', 't108586']",4,3,0,7079255,1267701,1443262,READY
t31406,2568,"['t31398', 't31400', 't31401', 't31403', 't314...",3,3,0,2196525,7746621,9751794,READY
...,...,...,...,...,...,...,...,...,...
t6342,516,['t6339'],8,2,0,2059335,1242559,10033757,READY
t100030,8268,"['t100024', 't100025', 't100027', 't100028', '...",1,4,0,1061185,7594966,9182334,READY
t5834,475,"['t5822', 't5828']",3,1,0,4288331,6675002,2350572,READY


## Step 4 : DDT


### Step 4.1: Initializing The tree


In [13]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            # self.weights = nn.Parameter(torch.zeros(num_input))
            self.weights = nn.Parameter(torch.empty(
                num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if a < 0.1:
            val = 1 - val
        if val >= 0.5:

            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

## Step 5: RL


In [14]:
def calc_execution_time(device, task, core, dvfs):
    if device['id'] == "cloud":
        return task["computationalLoad"] / device["voltages_frequencies"][0]
    else:
        return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device, task, core, dvfs):
    if device['id'] == "cloud":
        return 13.85 * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * device["voltages_frequencies"][core][dvfs][0]
    )
def calc_energy(device, task, core, dvfs):
    return calc_execution_time(device, task, core, dvfs) * calc_power_consumption(device, task, core, dvfs)


def calc_total(device, task, core, dvfs):
    timeTransMec = 0
    timeTransCC = 0
    exeTime = 0
    e = 0

    transferRate5g =1e9
    latency5g=5e-3
    transferRateFiber =1e10
    latencyFiber=1e-3

    timeDownMec = task["returnDataSize"] / transferRate5g
    timeDownMec += latency5g
    timeUpMec = task["dataEntrySize"] / transferRate5g
    timeUpMec += latency5g

    alpha = 52e-5
    beta = 3.86412
    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / transferRateFiber
    timeDownCC += latencyFiber
    timeUpCC = task["dataEntrySize"] / transferRateFiber
    timeUpCC += latencyFiber

    powerCC = 3.65 


    if device["id"].startswith("mec"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec *  timeTransMec
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime + timeTransMec 
        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  e + energyTransMec

    elif device['id'].startswith("cloud"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec * timeTransMec
        
        timeTransCC = timeUpCC+timeDownCC
        energyTransCC =  powerCC * timeTransCC
        
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime =  exeTime + timeTransMec +timeTransCC

        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  + energyTransMec + energyTransCC

    elif device['id'].startswith("iot"):
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime
        e = calc_energy(device, task, core, dvfs)
        totalEnergy = e

    return totalTime , totalEnergy

In [15]:
def checkIfSuitable(state, device):
    punishment = 0
    safeFail = 0
    taskFail = 0
    if  state['safe'] and not device["handleSafeTask"]:
        punishment += 25
        safeFail += 1
        
    if state['kind'] not in device["acceptableTasks"]:
        punishment += 25
        taskFail += 1
    return (20 if punishment > 0 else 0, taskFail, safeFail)

In [16]:
def getSetup(t, e, setup, alpha=1, beta=1):
    match setup:
        case "00":
            reward = -1 * (e + t)
        case "01":
            reward = -1 * (alpha * e + beta * t)
        case "02":
            reward = -1 / (e + t)
        case "03":
            reward = -1 / (alpha * e + beta * t)
        case "04":
            reward = -np.exp(e) - np.exp(t)
        case "05":
            reward = -np.exp(alpha * e) - np.exp(beta * t)
        case "06":
            reward = -np.exp(t + e)
        case "07":
            reward = -np.exp(alpha * t + beta * e)
        case "08":
            reward = np.exp(-t - e)
        case "09":
            reward = np.exp(-1 * (alpha * t + beta * e))
    
    return reward

In [17]:
tasks_copy = tasks.copy()
tasks_copy = tasks_copy.drop(["job","dependency","mobility","status"],axis=1)
taskList = tasks_copy.index.tolist()

In [18]:
class Environment:
    def __init__(self):
        self.totalSafeFail = 0
        self.totalTaskFail = 0
        self.totalReward = 0
        self.feature_size = 5
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions,
                         depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self, state, action):
        taskList.pop(0)
        device = devices.iloc[action]        

        punishment, taskFail, safeFail = checkIfSuitable(state, device)
        if safeFail:
            self.totalSafeFail += 1
        if taskFail:
            self.totalTaskFail += 1


        if not (punishment):
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] == 0:
                    total_t, total_e  = calc_total(device, state, coreIndex,np.random.randint(0,3))
                    reward = getSetup(total_t, total_e, "04")
                    # reward = -1 / (total_t + total_e)
                    # print(f"device {device['id']} ////// time {total_t}  ////// energy {total_e} ")
                    env.totalReward += reward
                    return (tasks_copy.loc[taskList[0]], reward, total_t, total_e)
        self.totalFail += 1
        return (tasks_copy.loc[taskList[0]], punishment, 0, 0)


    def train(self, num_epoch, num_episodes):

        total_avg_t = 0
        total_avg_e = 0
        total_avg_r = 0
        total_avg_l = 0

        for i in range(num_epoch):
            total_loss = 0
            env.totalFail = 0
            env.totalTaskFail = 0
            env.totalSafeFail = 0
            env.totalReward = 0
            total_loss = 0
            total_reward = 0
            totalTime = 0
            totalEnergy = 0


            
            for j in range(num_episodes):
                state = tasks_copy.loc[taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                
                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)
                action_index = torch.multinomial(action_probabilities, 1).item()

                next_state, reward, t, e = self.execute_action(state, action_index)
                loss = (output[action_index] * reward)

                total_reward += reward
                total_loss += loss
                totalTime += t
                totalEnergy += e
                
                

            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            avg_time = totalTime / num_episodes
            avg_energy = totalEnergy / num_episodes
            
            # avg_reward = total_reward / num_episodes
            avg_reward = env.totalReward / num_episodes
            avg_loss = total_loss/num_episodes


            total_avg_t += avg_time
            total_avg_e += avg_energy
            total_avg_l += avg_loss
            total_avg_r += avg_reward
            

            avg_loss.backward()
            self.optimizer.step()
            if i % 100 == 0:
                # print(f"Epoch {i+1} // avg cc time: {avg_cc} // avg mec: {avg_mec} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                # print(f"Epoch {i+1} // avg time: {avg_time} // avg added Time: {avg_added_time} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                print(f"Epoch {i+1}  // safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")
            
            # if i == 10001:
                
            #     print(f"safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")

                # env.totalFail = 0
                env.totalSafeFail = 0
                env.totalTaskFail

        avg_avg_t = total_avg_t / num_epoch
        avg_avg_l = total_avg_l / num_epoch
        avg_avg_r = total_avg_r / num_epoch
        avg_avg_e = total_avg_e / num_epoch


        print("===============================================================================")
        print(f'Overall Average Time across all epochs: {avg_avg_t}')
        print(f'Overall Average e across all epochs: {avg_avg_e:.2f}')
        print(f'Overall Average l across all epochs: {avg_avg_l:.2f}')
        print(f'Overall Average r across all epochs: {avg_avg_r:.2f}')
     





# env.totalAddedAvg += avg_cc


env = Environment()
tree = env.agent
env.train(1001, 10)

print('///////////////////')

for name, param in env.agent.named_parameters():
    if "prob_dist" or "bias" not in name:
        # print(name,param)
        pass

Epoch 1  // safe/task fail: 1/0 // Average Loss: 0.00 // Total Reward: -20.31 // Average Reward: -2.03 // Avg time: 0.15 // Avg energy: 0.05
Epoch 101  // safe/task fail: 0/0 // Average Loss: -0.07 // Total Reward: -21.43 // Average Reward: -2.14 // Avg time: 0.07 // Avg energy: 0.06
Epoch 201  // safe/task fail: 1/0 // Average Loss: -0.43 // Total Reward: -22.08 // Average Reward: -2.21 // Avg time: 0.25 // Avg energy: 0.04
Epoch 301  // safe/task fail: 0/0 // Average Loss: -1.13 // Total Reward: -22.98 // Average Reward: -2.30 // Avg time: 0.20 // Avg energy: 0.04
Epoch 401  // safe/task fail: 0/0 // Average Loss: -1.45 // Total Reward: -24.41 // Average Reward: -2.44 // Avg time: 0.25 // Avg energy: 0.05
Epoch 501  // safe/task fail: 0/1 // Average Loss: -3.30 // Total Reward: -20.26 // Average Reward: -2.03 // Avg time: 0.15 // Avg energy: 0.03
Epoch 601  // safe/task fail: 0/1 // Average Loss: -2.88 // Total Reward: -20.72 // Average Reward: -2.07 // Avg time: 0.19 // Avg energy: 